In [1]:
import numpy as np
import pandas as pd

import datetime

import warnings
warnings.filterwarnings('ignore')

/Users/naoki/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/naoki/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [13]:
h_trs = pd.read_csv("../input/historical_transactions.csv")
n_trs = pd.read_csv("../input/new_merchant_transactions.csv")

usecol = ["merchant_id", "merchant_group_id", "most_recent_sales_range", "category_4"]
merchant = pd.read_csv("../input/merchants.csv", usecols=usecol)

In [14]:
merchant["most_recent_sales_range"] = merchant.most_recent_sales_range.map(
    {
        "A": -0.3755114827304539, 
        "B": -0.1418195536770137, 
        "C": -0.1810503991535556, 
        "D": 0.9603337574351729, 
        "E": -0.22889415456488432

    }
)

In [15]:
trs = pd.concat([h_trs, n_trs], axis=0)

In [16]:
trs.purchase_date = pd.to_datetime(trs.purchase_date)

In [17]:
trs.purchase_date = trs.purchase_date.apply(lambda x:x.date())

In [18]:
trs["latest"] = (datetime.date(2018, 5, 1) - trs.purchase_date).dt.days

In [19]:
trs_concated = pd.merge(trs, merchant, on="merchant_id", how="left")

In [20]:
trs_concated.head()

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,latest,merchant_group_id,most_recent_sales_range,category_4
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25,1.0,16,37,310,35.0,-0.375511,Y
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15,1.0,16,16,290,2084.0,-0.375511,Y
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09,1.0,16,37,265,27369.0,-0.181050,Y
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02,1.0,16,34,241,24104.0,0.960334,Y
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10,1.0,16,37,417,35.0,-0.375511,Y


In [21]:
def sig(x):
    return 1.0/(1.0+np.exp(-x))

# Group

In [10]:
df_group = trs_concated[["merchant_group_id", "merchant_category_id", "subsector_id", "purchase_amount"]]

In [11]:
df_group_perchase = df_group.groupby(["merchant_group_id", "merchant_category_id", "subsector_id"])["purchase_amount"].mean().reset_index()
df_group_perchase.columns = ["merchant_group_id", "merchant_category_id", "subsector_id", "sig_amount_per_group"]
df_group_perchase.sig_amount_per_group = df_group_perchase.sig_amount_per_group.apply(sig)
df_group_perchase

,merchant_group_id,merchant_category_id,subsector_id,sig_amount_per_group
0,1.0,774,29,0.345684
1,2.0,307,19,0.339600
2,2.0,414,29,0.336388
3,3.0,793,38,0.365701
4,4.0,63,27,0.334184
5,4.0,705,33,0.340470
6,5.0,68,27,0.400038
7,5.0,130,41,0.331407
8,5.0,178,29,0.343162
9,5.0,180,20,0.328915


# Category

In [12]:
df_category = trs_concated[["category_1", "category_2", "category_3", "category_4", "purchase_amount"]]

In [13]:
df_category_perchase = df_category.groupby(["category_1", "category_2", "category_3", "category_4"])["purchase_amount"].mean().reset_index()
df_category_perchase.columns = ["category_1", "category_2", "category_3", "category_4", "sig_amount_per_category"]
df_category_perchase.sig_amount_per_category = df_category_perchase.sig_amount_per_category.apply(sig)

In [14]:
df_category_perchase

,category_1,category_2,category_3,category_4,sig_amount_per_category
0,N,1.0,A,N,0.468163
1,N,1.0,A,Y,0.728098
2,N,1.0,B,N,0.403196
3,N,1.0,B,Y,0.373990
4,N,1.0,C,N,0.483706
5,N,1.0,C,Y,0.542320
6,N,2.0,A,N,0.463504
7,N,2.0,A,Y,0.457174
8,N,2.0,B,N,0.450150
9,N,2.0,B,Y,0.385218


# District

In [15]:
df_district = trs_concated[["city_id", "state_id", "purchase_amount"]]

In [16]:
df_district_perchase = df_district.groupby(["city_id", "state_id"])["purchase_amount"].mean().reset_index()
df_district_perchase.columns = ["city_id", "state_id", "sig_amount_per_district"]
df_district_perchase.sig_amount_per_district =  df_district_perchase.sig_amount_per_district.apply(sig)

In [17]:
df_district_perchase

,city_id,state_id,sig_amount_per_district
0,-1,-1,0.414542
1,1,12,0.505768
2,2,11,0.374538
3,3,16,0.459648
4,4,10,0.504751
5,6,9,0.357629
6,7,9,0.394438
7,8,12,0.355842
8,9,16,0.363492
9,10,3,0.418056


In [18]:
def concat(df_base, df_add, on_col):
    df_result = pd.merge(df_base, df_add, on=on_col, how="left")
    return df_result

In [19]:
trs_concated1 = concat(trs_concated, df_group_perchase, ["merchant_group_id", "merchant_category_id", "subsector_id"])

for col in ["merchant_group_id", "merchant_category_id", "subsector_id"]:
    del trs_concated1[col]

In [20]:
trs_concated2 = concat(trs_concated1, df_category_perchase, ["category_1", "category_2", "category_3", "category_4"])

for col in ["category_1", "category_2", "category_3", "category_4"]:
    del trs_concated2[col]

In [21]:
trs_concated3 = concat(trs_concated2, df_district_perchase, ["city_id", "state_id"])

for col in ["city_id", "state_id"]:
    del trs_concated3[col]

In [22]:
df_trs = trs_concated3.drop(["merchant_id", "installments", "month_lag", "purchase_date"], axis=1)

In [23]:
df_trs2 = df_trs.copy()

In [24]:
df_trs.authorized_flag = df_trs.authorized_flag.map({"N": -1, "Y": 1})
df_trs.latest = df_trs.latest.apply(lambda x:1.0/x)
df_trs.avg_sales_lag3 = df_trs.avg_sales_lag3.apply(sig)
df_trs.purchase_amount = df_trs.purchase_amount.apply(sig)

In [25]:
df_trs["fifa_score"] = 100000 * df_trs.authorized_flag * df_trs.purchase_amount * df_trs.latest * df_trs.avg_sales_lag3 * df_trs.sig_amount_per_group * df_trs. sig_amount_per_category * df_trs.sig_amount_per_district

In [26]:
df_trs_fifa = df_trs.groupby("card_id")["fifa_score"].sum().reset_index()

In [41]:
df_trs_fifa.columns = ["card_id", "fifa_score"]

In [43]:
df_trs_fifa.to_csv("../input/fifa_score.csv", index=False)